In [1]:
import os
import gensim
from tqdm import tqdm
import numpy as np
import keras
import tensorflow as tf

2022-07-26 11:50:39.897246: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-07-26 11:50:39.900081: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-26 11:50:39.900091: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from nltk.stem import WordNetLemmatizer
from collections import Counter

In [3]:
print(os.listdir("word2vec_dataset/word2vec_dataset/review_hotels/data"))

['chicago', 'las-vegas', 'london', 'dubai', 'new-york-city', 'shanghai', 'new-delhi', 'san-francisco', 'montreal', '.DS_Store', 'beijing']


In [4]:
alltext=[]
path="word2vec_dataset/word2vec_dataset/review_hotels/data"
for folders in os.listdir(path):
    if not folders[0].isalnum():
        continue
    for filename in os.listdir(path + '/' + folders):
        with open(os.path.join(path + '/' + folders,filename),'r',errors='ignore') as f:
            for sentence in f.readlines():
                alltext.append(sentence)

In [5]:
alltext[0][alltext[0].index('\t')+1:]

"Ups and Downs\tI was in Chicago for business and stayed with another colleague in a room. The room was very clean, but the hotel is a bit cheap when it comes to bathroom amenities. There were two of us booked in the room--why only one bath robe? Turn down service was spotty. One night we had a drink in the room; our wine glasses were taken but never returned until we called. There is a service called &quot;Delighted to Service&quot;----the person who answered when I called apparently wasn't told you were supposed to be friendly when a hotel guest calls!The bar in the hotel, Novo is a complete rip-off. One night 4 of us met for drinks. 2 of my friends ordered the special martinis at $11 each. When the first one arrived, we laughed and joked that the waitress must have been thristy since it wasn't even 3/4 full; the second maritini was also less then 3/4 full so we joked that the waitress must have tripped and spilled some. The friend who ordered an expensive wine got about 2 inches of 

In [6]:
n=len(alltext)
no_date_text=[]
for i in range(n):
    try:
        no_date_text.append(alltext[i][alltext[i].index('\t')+1:])
    except:
        continue

In [7]:
import re
import string
import nltk

stopwords = nltk.corpus.stopwords.words('english')
lemma=WordNetLemmatizer()

In [8]:
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [lemma.lemmatize(word) for word in tokens if word not in stopwords and word.isalpha()]
    return text

In [9]:
word_lists = []
all_words=Counter()
maxlen=30

for i,text in tqdm(enumerate(no_date_text)):
    no_date_text[i] = clean_text(text)[:maxlen]
    for word in no_date_text[i]:
        all_words[word]+=1

255138it [01:38, 2581.10it/s]


In [10]:
k=0
uniq_words=[]
for w,i in sorted(all_words.items(),key=lambda x:x[1],reverse=True):
    if k>9999:
        break
    uniq_words.append(w)
    k+=1

In [11]:
unique_word_dict = {}
for i, word in tqdm(enumerate(uniq_words)):
    unique_word_dict.update({
        word: i
    })

10000it [00:00, 2323585.40it/s]


In [12]:
for w,i in unique_word_dict.items():
    if i>20:
        break
    print(w,i)

hotel 0
room 1
great 2
location 3
stayed 4
stay 5
night 6
good 7
staff 8
clean 9
nice 10
time 11
one 12
service 13
bed 14
place 15
small 16
excellent 17
booked 18
friendly 19
london 20


In [13]:
window=2

word_lists = []

for j in tqdm(range(len(no_date_text))):
    no_date_text[j]=[word for word in no_date_text[j] if word in uniq_words]
    for i, word in enumerate(no_date_text[j]):
        for w in range(window):
            if i + 1 + w < len(no_date_text[j]): 
                word_lists.append([word] + [no_date_text[j][(i + 1 + w)]])    
            if i - w - 1 >= 0:
                word_lists.append([word] + [no_date_text[j][(i - w - 1)]])

100%|█████████████████████████████████| 255138/255138 [01:24<00:00, 3015.18it/s]


In [14]:
len(word_lists)

24557670

In [15]:
n=len(unique_word_dict)
n

10000

In [16]:
from tensorflow.sparse import SparseTensor as spa

In [17]:
x=spa(indices=[[i,unique_word_dict[word_lists[i][0]]] for i in range(128)],values=[1 for i in range(128)],dense_shape=[128,n])

2022-07-26 11:54:03.751229: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-07-26 11:54:03.751265: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-26 11:54:03.751287: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (localhost.localdomain): /proc/driver/nvidia/version does not exist
2022-07-26 11:54:03.751987: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
print(x.shape)

(128, 10000)


In [19]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, word_lists, unique_word_dict, batch_size=128,shuffle=False):
        'Initialization'
        self.word_lists = word_lists
        self.batch_size = batch_size
        self.unique_word_dict = unique_word_dict
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.word_lists) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.word_lists[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.word_lists))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, word_lists_temp):
        'Generates data containing batch_size samples'
        # Initialization
        n=len(self.unique_word_dict)
        n1=len(word_lists_temp)
        
        X = spa(indices=[[i,self.unique_word_dict[word_lists_temp[i][0]]] for i in range(n1)],values=[1 for i in range(n1)],dense_shape=[n1,n])
        Y = spa(indices=[[i,self.unique_word_dict[word_lists_temp[i][1]]] for i in range(n1)],values=[1 for i in range(n1)],dense_shape=[n1,n])

        #for i, word_list in enumerate(word_lists_temp):

            #X_row = np.zeros(n)
            #Y_row = np.zeros(n)

            #X_row[self.unique_word_dict[word_list[0]]] = 1

            #Y_row[self.unique_word_dict[word_list[1]]] = 1
            #X_row=spa(indices=[[self.unique_word_dict[word_list[i][0]] for i in range(n1)]],values=[1 for i in range(n1)],dense_shape=[n1,n])
            #Y_row=spa(indices=[[self.unique_word_dict[word_list[1]]]],values=[1],dense_shape=[n])

            #X.append(X_row)
            #Y.append(Y_row)

        return X,tf.sparse.to_dense(Y)

In [20]:
from keras.models import Model
from keras.layers import Dense, Input

inp_out_nodes=len(unique_word_dict)
embed_size = 25

data_generator=DataGenerator(word_lists,unique_word_dict,batch_size=128)

inp = Input(shape=(inp_out_nodes,),sparse=True)
x = Dense(units=embed_size, activation='linear')(inp)
x = Dense(units=inp_out_nodes, activation='softmax')(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

model.fit_generator(
    generator=data_generator,
    use_multiprocessing=True,
    workers=6,
    epochs=1
    )


/tmp/ipykernel_40264/11246289.py:15: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(
/home/aditya/.local/lib/python3.10/site-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/dense/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model/dense/embedding_lookup_sparse/Reshape:0", shape=(None, 25), dtype=float32), dense_shape=Tensor("gradient_tape/model/dense/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


191856/191856 [==============================] - 3176s 17ms/step - loss: 6.6192


In [21]:
weights = model.get_weights()[0]

embedding_dict = {}
for word in uniq_words: 
    embedding_dict.update({
        word: weights[unique_word_dict.get(word)]
        })

In [22]:
model.save('w2v_w5model')

INFO:tensorflow:Assets written to: w2v_w5model/assets


INFO:tensorflow:Assets written to: w2v_w5model/assets


In [22]:
pol=embedding_dict['polite']
dirt=embedding_dict['dirty']
fra=embedding_dict['france']
sho=embedding_dict['shocked']

In [23]:
from numpy.linalg import norm
def cossim(x,y):
    return np.dot(x,y)/norm(x)*norm(y)
def top(x):
    tops=[]
    for w,i in embedding_dict.items():
        tops.append([cossim(i,x),w])
    tops.sort(reverse=True)
    return tops[:10]

In [24]:
print(top(pol))

[[6.1565533, 'polite'], [5.936247, 'courteous'], [5.7218723, 'informative'], [5.7175884, 'smiley'], [5.6953373, 'professional'], [5.6798983, 'freindly'], [5.6762376, 'friendly'], [5.6517563, 'personable'], [5.6475177, 'courtious'], [5.6463637, 'accommodating']]


In [25]:
print(top(dirt))

[[6.0093656, 'dirty'], [5.709279, 'filthy'], [5.5923386, 'smelly'], [5.5772266, 'moldy'], [5.5136285, 'unclean'], [5.4892893, 'stinky'], [5.370372, 'gross'], [5.3120003, 'carpet'], [5.310087, 'hole'], [5.306463, 'stained']]


In [26]:
print(top(fra))

[[2.2151058, 'france'], [1.9428515, 'rajasthan'], [1.9242859, 'ireland'], [1.923313, 'india'], [1.9155867, 'sydney'], [1.9119023, 'sonoma'], [1.9090952, 'canada'], [1.8196089, 'kuoni'], [1.8059552, 'edinburgh'], [1.8055127, 'usa']]


In [27]:
print(top(sho))

[[1.760999, 'shocked'], [1.5220942, 'horrified'], [1.3938911, 'heavily'], [1.3780063, 'already'], [1.3669505, 'suspicious'], [1.3649032, 'unfortunately'], [1.3604099, 'dissapointed'], [1.3457959, 'initially'], [1.3436953, 'laughed'], [1.3413905, 'saw']]


In [7]:
for i in range(len(no_date_text)):
    no_date_text[i]=gensim.utils.simple_preprocess(no_date_text[i])

In [8]:
no_date_text[0]

['ups',
 'and',
 'downs',
 'was',
 'in',
 'chicago',
 'for',
 'business',
 'and',
 'stayed',
 'with',
 'another',
 'colleague',
 'in',
 'room',
 'the',
 'room',
 'was',
 'very',
 'clean',
 'but',
 'the',
 'hotel',
 'is',
 'bit',
 'cheap',
 'when',
 'it',
 'comes',
 'to',
 'bathroom',
 'amenities',
 'there',
 'were',
 'two',
 'of',
 'us',
 'booked',
 'in',
 'the',
 'room',
 'why',
 'only',
 'one',
 'bath',
 'robe',
 'turn',
 'down',
 'service',
 'was',
 'spotty',
 'one',
 'night',
 'we',
 'had',
 'drink',
 'in',
 'the',
 'room',
 'our',
 'wine',
 'glasses',
 'were',
 'taken',
 'but',
 'never',
 'returned',
 'until',
 'we',
 'called',
 'there',
 'is',
 'service',
 'called',
 'quot',
 'delighted',
 'to',
 'service',
 'quot',
 'the',
 'person',
 'who',
 'answered',
 'when',
 'called',
 'apparently',
 'wasn',
 'told',
 'you',
 'were',
 'supposed',
 'to',
 'be',
 'friendly',
 'when',
 'hotel',
 'guest',
 'calls',
 'the',
 'bar',
 'in',
 'the',
 'hotel',
 'novo',
 'is',
 'complete',
 'rip',
 

In [9]:
w2v_model = gensim.models.Word2Vec(no_date_text,
                                   vector_size=100,
                                   window=5,
                                   min_count=2)

In [25]:
w2v_model.wv.most_similar('polite')

[('courteous', 0.9562245607376099),
 ('curtious', 0.9254128932952881),
 ('cordial', 0.9228877425193787),
 ('curteous', 0.9179827570915222),
 ('friendly', 0.8865291476249695),
 ('professional', 0.8803302645683289),
 ('personable', 0.880230188369751),
 ('obliging', 0.876288115978241),
 ('freindly', 0.8689395785331726),
 ('attentive', 0.8673774600028992)]

In [10]:
w2v_model.wv.most_similar('polite')

[('courteous', 0.9403677582740784),
 ('cordial', 0.9175040125846863),
 ('curtious', 0.8974000811576843),
 ('curteous', 0.894274890422821),
 ('friendly', 0.8864861130714417),
 ('personable', 0.8689129948616028),
 ('courtious', 0.8669386506080627),
 ('professional', 0.8653585314750671),
 ('freindly', 0.8647214770317078),
 ('attentive', 0.8505169153213501)]

In [26]:
w2v_model.wv.most_similar('dirty')

[('filthy', 0.8850131034851074),
 ('unclean', 0.8443936109542847),
 ('grubby', 0.8276762366294861),
 ('dusty', 0.8143419027328491),
 ('smelly', 0.8079279065132141),
 ('stained', 0.7973206639289856),
 ('dingy', 0.7956437468528748),
 ('grimy', 0.7954571843147278),
 ('threadbare', 0.7616309523582458),
 ('grungy', 0.7595184445381165)]

In [11]:
w2v_model.wv.most_similar('dirty')

[('filthy', 0.8956348299980164),
 ('unclean', 0.8347494006156921),
 ('dusty', 0.8254992961883545),
 ('grubby', 0.8135543465614319),
 ('stained', 0.8135532736778259),
 ('smelly', 0.8127628564834595),
 ('dingy', 0.8025403022766113),
 ('grimy', 0.7875556349754333),
 ('gross', 0.769194483757019),
 ('mouldy', 0.7645899057388306)]

In [27]:
w2v_model.wv.most_similar('france')

[('germany', 0.7601078152656555),
 ('hawaii', 0.7335811257362366),
 ('barcelona', 0.7212272882461548),
 ('edinburgh', 0.7177729606628418),
 ('manchester', 0.7142756581306458),
 ('canada', 0.7137922048568726),
 ('mexico', 0.7136322259902954),
 ('greece', 0.7028810381889343),
 ('rome', 0.6963257193565369),
 ('england', 0.694165050983429)]

In [12]:
w2v_model.wv.most_similar('france')

[('canada', 0.7614089846611023),
 ('hawaii', 0.7559431195259094),
 ('germany', 0.7554485201835632),
 ('russia', 0.7344716787338257),
 ('spain', 0.7289267778396606),
 ('detroit', 0.7277833223342896),
 ('boston', 0.7261947393417358),
 ('manchester', 0.7240197062492371),
 ('los', 0.7203864455223083),
 ('ottawa', 0.7149811387062073)]

In [28]:
w2v_model.wv.most_similar('shocked')

[('dismayed', 0.8659629821777344),
 ('horrified', 0.8553714752197266),
 ('appalled', 0.8544608950614929),
 ('astonished', 0.8502187132835388),
 ('amazed', 0.8499859571456909),
 ('suprised', 0.8346281051635742),
 ('surprised', 0.8211231231689453),
 ('stunned', 0.818147599697113),
 ('surprized', 0.8072366118431091),
 ('astounded', 0.7653063535690308)]

In [13]:
w2v_model.wv.most_similar('shocked')

[('astonished', 0.8625437617301941),
 ('appalled', 0.8525459170341492),
 ('horrified', 0.850727915763855),
 ('amazed', 0.8442299365997314),
 ('stunned', 0.834455132484436),
 ('dismayed', 0.8310640454292297),
 ('suprised', 0.804319441318512),
 ('surprised', 0.7833690643310547),
 ('astounded', 0.7687271237373352),
 ('surprized', 0.7598392963409424)]

In [14]:
w2v_model.wv['polite']

array([-1.1988171 , -0.9674909 ,  0.64097   , -0.71369845, -0.00628295,
        2.1619508 ,  1.5266845 ,  0.24826671, -3.128817  , -3.4203453 ,
       -2.6994023 ,  1.4151658 ,  3.382561  ,  0.8212804 , -1.8087522 ,
        0.20338462,  0.51118785,  0.16701764,  1.3731269 ,  1.0951844 ,
        2.220896  ,  1.8583843 ,  1.3731807 ,  0.72523826, -1.4466767 ,
        0.43909648, -2.9454157 ,  0.00636258, -0.87504333,  1.357516  ,
        0.88279504,  3.304704  ,  1.71506   , -0.7912113 , -1.3957821 ,
        1.512245  , -0.9164039 , -1.5361773 , -1.6478198 , -0.00931805,
        0.2834988 , -1.0733864 ,  0.45020404,  2.9922414 ,  2.6350858 ,
       -1.2810483 ,  3.0789678 , -4.661094  ,  2.473212  , -1.2049787 ,
        2.6635103 ,  1.3205016 ,  1.3404276 ,  2.5705771 , -2.1999757 ,
       -2.4327166 , -1.8434448 , -0.7479542 , -3.8086164 , -0.4265809 ,
       -2.3046997 , -4.9821773 ,  2.0753124 ,  0.43855223,  0.84447545,
        1.0311258 , -1.1947426 , -0.35087544, -0.22979113, -0.41